# numba

In [39]:
import numpy as np
from numba import config, njit, prange
import time
import numba

def isArraySorted(arr):
    n = len(arr)
    if (n > 0):
        prev = arr[0]
        for i in range(n):
            if (arr[i] < prev):
                return False
    return True;

def createRandomArr(n):
    return np.random.randint(0, 100, n)

N = 10000
ITERS = 100

@njit
def quickSortSeq(arr, low, high):
    if (low < high):
        # partitionHoar
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while (1):
            while (arr[i] < pivot):
                i = i + 1
            while (arr[j] > pivot):
                j = j - 1

            if (i >= j):
                break

            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        # recurcive call
        quickSortSeq(arr, low, pi)
        quickSortSeq(arr, pi + 1, high)

@njit(parallel=True)
def quickSortPar(arr, low, high):
    if low < high:
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while True:
            while arr[i] < pivot:
                i = i + 1
            while arr[j] > pivot:
                j = j - 1

            if i >= j:
                break

            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        for id in prange(2):
            if (id == 0):
                quickSortSeq(arr, low, pi)
            else:
                quickSortSeq(arr, pi + 1, high)

In [40]:
arr = createRandomArr(10)
print(arr)

[17 51 92  8 33 94  3 82 36 45]


In [41]:
quickSortPar(arr, 0, len(arr)-1)
print(arr)

[ 3  8 17 33 36 45 51 82 92 94]


In [42]:
def calcParTime1():
    arr = createRandomArr(10)
    quickSortPar(arr, 0, len(arr) - 1)
    threads_n = 4
    while (threads_n > 0):
        numba.set_num_threads(threads_n)
        full_time = 0
        for it in range(ITERS):
            arr = createRandomArr(N)
            # print(arr)
    
            start_time = time.time()
    
            quickSortPar(arr, 0, len(arr) - 1)
    
            end_time = time.time()
            elapsed_time = end_time - start_time;
            # print(elapsed_time)
            full_time += elapsed_time
    
            # print(arr)
            if (not isArraySorted(arr)):
                print("Massive is not sorted\n")
            
        full_time /= ITERS
        print(f"Time taken ({threads_n} threads):", full_time, "seconds")
        threads_n //= 2

In [48]:
N = 10000
ITERS = 100

config.THREADING_LAYER = 'omp'
config.NUMBA_DEFAULT_NUM_THREADS = 4

for i in range(3):
    calcParTime1()

Time taken (4 threads): 0.0005791831016540527 seconds
Time taken (2 threads): 0.00038124799728393553 seconds
Time taken (1 threads): 0.0003824925422668457 seconds
Time taken (4 threads): 0.00037195682525634763 seconds
Time taken (2 threads): 0.0003602170944213867 seconds
Time taken (1 threads): 0.00037729740142822266 seconds
Time taken (4 threads): 0.0003763461112976074 seconds
Time taken (2 threads): 0.00038551807403564454 seconds
Time taken (1 threads): 0.00039507389068603514 seconds


# numba.openmp

In [49]:
from numba import njit
from numba.openmp import openmp_context as openmp
from numba.openmp import omp_get_thread_num, omp_get_num_threads, omp_set_num_threads, omp_get_max_threads, omp_get_wtime
import numpy as np
from numba import njit
from numba.openmp import openmp_context as openmp
import random

@njit
def quickSortPar(arr, low, high, max_d, d = 0):
    if (low < high):
        # partitionHoar
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while (1):
            while (arr[i] < pivot):
                i = i + 1
            while (arr[j] > pivot):
                j = j - 1
        
            if (i >= j):
                break
        
            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        # recurcive call
        if (d < max_d):
            #with openmp("parallel sections"):
            with openmp("task shared(arr, low, pi)"):
                quickSortPar(arr, low, pi, max_d, d + 1)
                #quickSortSeq(arr, low, pi)
            with openmp("task shared(arr, pi, high)"):    
                quickSortPar(arr, pi + 1, high, max_d, d + 1)  
                #quickSortSeq(arr, pi + 1, high)
        else:
            quickSortPar(arr, low, pi, max_d, d + 1)
            quickSortPar(arr, pi + 1, high, max_d, d + 1)
        #quickSortPar(arr, low, pi)
        #quickSortPar(arr, pi + 1, high)

In [51]:
arr = createRandomArr(10)
print(arr)
with openmp("parallel"):
    with openmp("single"):
        quickSortPar(arr, 0, len(arr)-1, 2)
print(arr)

[ 7 72 72 88 57 58 87 25 26 80]
[ 7 25 26 57 58 72 72 80 87 88]


In [60]:
@njit
def setNumThreads(n):
    omp_set_num_threads(n)


def calcParTime2():
    arr = createRandomArr(10)
    with openmp("parallel"):
        with openmp("single"):
            quickSortPar(arr, 0, len(arr)-1, 2)
    threads_n = 4
    while (threads_n > 0):
        setNumThreads(threads_n)
        full_time = 0
        for it in range(ITERS):
            arr = createRandomArr(N)
            # print(arr)

            max_d = 0
            sum_th_n = 1
            cur_th_n = 1
            while (sum_th_n < threads_n):
                cur_th_n *= 2
                sum_th_n += cur_th_n
                max_d += 1
    
            start_time = time.time()
    
            #quickSortPar(arr, 0, len(arr) - 1)
            with openmp("parallel"):
                with openmp("single"):
                    quickSortPar(arr, 0, len(arr)-1, max_d)
    
            end_time = time.time()
            elapsed_time = end_time - start_time;
            # print(elapsed_time)
            full_time += elapsed_time
    
            # print(arr)
            if (not isArraySorted(arr)):
                print("Massive is not sorted\n")
            
        full_time /= ITERS
        print(f"Time taken ({threads_n} threads):", full_time, "seconds")
        threads_n //= 2

In [62]:
N = 10000
ITERS = 100

config.THREADING_LAYER = 'omp'
config.NUMBA_DEFAULT_NUM_THREADS = 4

for i in range(3):
    calcParTime2()

Time taken (4 threads): 0.0005464339256286621 seconds
Time taken (2 threads): 0.0004477691650390625 seconds
Time taken (1 threads): 0.00044672250747680665 seconds
Time taken (4 threads): 0.00043246984481811525 seconds
Time taken (2 threads): 0.0004343700408935547 seconds
Time taken (1 threads): 0.0004311656951904297 seconds
Time taken (4 threads): 0.00044522285461425783 seconds
Time taken (2 threads): 0.00044421911239624025 seconds
Time taken (1 threads): 0.00045875787734985353 seconds


# Testing

In [196]:
@njit
def getFreeThreads(threads):
    id1 = -1;
    id2 = -1;
    for i in range(len(threads)):
        if (threads[i]):
            if (id1 == -1):
                id1 = i
            else:
                id2 = i
        if (not id1 == -1 and not id2 == -1):
            break;
    return id1, id2
        

@njit
def quickSortPar(arr, low, high, threads):
    if (low < high):
        # partitionHoar
        i = low
        j = high
        pivot = arr[(i + j) // 2]
        while (1):
            while (arr[i] < pivot):
                i = i + 1
            while (arr[j] > pivot):
                j = j - 1
        
            if (i >= j):
                break
        
            arr[i], arr[j] = arr[j], arr[i]
            i = i + 1
            j = j - 1
        pi = j

        threadID = omp_get_thread_num()
        #print("start:", threadID)
        threads[threadID] = True;
        #print("HERE2", threads)
        id1, id2 = getFreeThreads(threads)
        #print(id1, id2)
        # recurcive call
        if (not id1 == -1 and not id2 == -1):
            threads[id1] = False
            threads[id2] = False
            #print("HERE", threads)
            with openmp("parallel shared(arr, low, pi, high, threads)"):
                threadID = omp_get_thread_num()
                #print(threadID)
                if (threadID == id1):
                    quickSortPar(arr, low, pi, threads)
                elif (threadID == id2):
                    quickSortPar(arr, low, pi, threads)
        else:
            quickSortPar(arr, low, pi, threads)
            quickSortPar(arr, pi + 1, high, threads)
        '''
        if (d < max_d):
            #with openmp("parallel"):
            with openmp("task shared(arr, low, pi)"):
                quickSortPar(arr, low, pi, max_d, d + 1)
                #quickSortSeq(arr, low, pi)
            with openmp("task shared(arr, pi, high)"):    
                quickSortPar(arr, pi + 1, high, max_d, d + 1)  
                #quickSortSeq(arr, pi + 1, high)
        else:
            quickSortPar(arr, low, pi, max_d, d + 1)
            quickSortPar(arr, pi + 1, high, max_d, d + 1)
        '''
        #quickSortPar(arr, low, pi)
        #quickSortPar(arr, pi + 1, high)